In [6]:
%pip install imblearn
%pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [8]:
file_path = r"/Users/anjanapro/Desktop/machine learning/flight_delay_prediction/notebooks/merged_data_with_labels.csv"
df = pd.read_csv(file_path)

In [9]:
# get class value counts to see if the data set is imbalanced
class_counts = df['ArrDel15'].value_counts()

# print the counts 
print(class_counts)

ArrDel15
0.0    1463378
1.0     388058
Name: count, dtype: int64


In [10]:
# assuming 'ArrDel15' is the target variable
# X = df.drop('ArrDel15', axis=1) 
# y = df['ArrDel15']  

# make a list of the desired columns in the data frame with a normal or skewed distribution from EDA
desired_columns_clf = ["windspeedKmph", "winddirDegree", "precipMM", "visibility", "pressure", "cloudcover", "DewPointF", "WindGustKmph", "tempF", "WindChillF", "humidity", "DepDelayMinutes", "ArrDelayMinutes", "ArrDel15"]

# filter the data frame with only the filtered columns by dropping the unrequired columns
df_filtered = df.drop(columns=list(set(df.columns) - set(desired_columns_clf)))

# split the data into training and testing sets 
X_train, X_test, y_train, y_test = train_test_split(df_filtered.drop("ArrDel15", axis=1), df_filtered["ArrDel15"], test_size=0.2, random_state=42)

# define and fit the SMOTE oversampler for classifier
smote = SMOTE(random_state=42)
X_resampled, y_resampled_clf = smote.fit_resample(X_train, y_train)

# drop the ArrDelayMinutes column from the Classification feature training set 
X_resampled_filter = X_resampled.drop('ArrDelayMinutes', axis=1) 
# Store that ArrDelayMinutes column so that there is a y_train of resampled data for the regression models later
y_train_reg = X_resampled["ArrDelayMinutes"]

# drop the ArrDelayMinutes column from the Classification feature testing set 
X_test_filter = X_test.drop('ArrDelayMinutes', axis=1) 
# Store that ArrDelayMinutes column so that there is a y_test of resampled data for the regression models later
y_test_reg = X_test["ArrDelayMinutes"]

clf_train_data = pd.concat([X_resampled_filter, y_resampled_clf], axis=1)
clf_test_data = pd.concat([X_test_filter, y_test], axis=1)

clf_train_data.to_csv('clf_train_data.csv', index=False)
clf_test_data.to_csv('clf_test_data.csv', index=False)

reg_train_data = pd.concat([X_resampled_filter, y_train_reg], axis=1)
reg_test_data = pd.concat([X_test_filter, y_test_reg], axis=1)

reg_train_data.to_csv('reg_train_data.csv', index=False)
reg_test_data.to_csv('reg_test_data.csv', index=False)
